In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.collections as collections
import pandas as pd
import seaborn as sns
import scipy

import pymc3 as pm
import theano
import theano.tensor as tt
from theano import tensor as T
import arviz as az

from NUTS_more_pars_helpers import generate_data, llik_td_vectorized, update_Q, theano_llik_td, get_nuts_est

## Parameter recoverability using NUTS with more pars

In [2]:
true_alpha_neg = .3
true_alpha_pos = .3
true_exp_neg = 1
true_exp_pos = 1
true_beta = 1.5
est_df = get_nuts_est(t_alpha_neg = true_alpha_neg, t_alpha_pos = true_alpha_pos, t_exp_neg = true_exp_neg, t_exp_pos = true_exp_pos, t_beta = true_beta, n=60)

ValueError: Cannot compute test value: input 0 (a) of Op Elemwise{lt,no_inplace}(a, b) missing default value.  
Backtrace when that variable is created:

  File "/Users/zeynepenkavi/anaconda/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3058, in run_cell_async
    interactivity=interactivity, compiler=compiler, result=result)
  File "/Users/zeynepenkavi/anaconda/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3249, in run_ast_nodes
    if (await self.run_code(code, result,  async_=asy)):
  File "/Users/zeynepenkavi/anaconda/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-d83c31aede13>", line 6, in <module>
    est_df = get_nuts_est(t_alpha_neg = true_alpha_neg, t_alpha_pos = true_alpha_pos, t_exp_neg = true_exp_neg, t_exp_pos = true_exp_pos, t_beta = true_beta, n=60)
  File "/Users/zeynepenkavi/Docum

In [ ]:
a,b = T.scalars('a', 'b')
x,y = T.matrices('x', 'y')

z_switch = T.switch(T.lt(a, b), T.mean(x), T.mean(y))

f_switch = theano.function([a, b, x, y], z_switch,
                           mode=theano.Mode(linker='vm'))

val1 = 0.
val2 = 1.
big_mat1 = np.ones((10000, 1000))
big_mat2 = np.ones((10000, 1000))

f_switch(val1, val2, big_mat1, big_mat2)

In [ ]:
a = f_switch(val1, val2, big_mat1, big_mat2)

In [ ]:
t,u = T.scalars('t', 'u')
zt = T.lt(t, u)
ft = theano.function([t, u], zt, mode =theano.Mode(linker='vm'))
ft(val1, 0)

Next steps:

Simulations:

-  Can pyMC recover parameters better for the whole range?
    -  Conclusion: Not necessarily but   
        - it keeps all values within bounds
        - it does not fail in any other way compared to the realistic MLE estimates (i.e. those starting from random values instead of the true values used for data generation)  
        
- Is recoverability worse for generating processes with more parameters
    - TODO: Write new/more generalizable ~~data generation~~ and par recovery functions for different RL models (with more parameters)
    - Compare parameter recoverability for at least one more RL model with more parameters 
    - Not interested in comparing MLE versus NUTS but compare simulation results of NUTS estimates between more simple and complicated processes
    
Fitting:

- Convert the recoverability function to fitting function

```
 # NUTS estimate
    actions_ = theano.shared(np.asarray(actions, dtype='int16'))
    with pm.Model() as m:
        alpha = pm.Beta('alpha', 1, 1)
        beta = pm.HalfNormal('beta', 10)
        like = pm.Potential('like', theano_llik_td(alpha, beta, machines, actions, rewards, n))
        tr = pm.sample()
    
    nuts_alpha_ave = np.mean(tr.alpha)
    nuts_beta_ave = np.mean(tr.beta)
    nuts_alpha_std = np.std(tr.alpha)
    nuts_beta_std = np.std(tr.beta)
    nuts_llik = llik_td_vectorized([nuts_alpha_ave, nuts_beta_ave], *(machines, actions, rewards))
```
- Fit to one subject's data (using the fitting function created above in fit_rl_cv.py)
- Fit to all subjects' data